# Air pollution estimation

In [13]:
import pandas as pd
import numpy as np
import os

from utils import *

datasets_folder = '.\datasets'

## Dataframe creation

### Air pollution Dataset

In [3]:
air_pollution_dataset = os.path.join(datasets_folder,'pollution\pollution.csv')

air_pollution_df = pd.read_csv(air_pollution_dataset, sep=';')
air_pollution_df

,COD_STAZ,AGENTE,DATA_INIZIO,DATA_FINE,VALORE,UM
0,PORTA SAN FELICE,NO2 (BIOSSIDO DI AZOTO),2019-01-01T17:01:00+01:00,2019-01-01T18:00:00+01:00,42.0,ug/m3
1,PORTA SAN FELICE,NO2 (BIOSSIDO DI AZOTO),2019-01-02T07:01:00+01:00,2019-01-02T08:00:00+01:00,42.0,ug/m3
2,PORTA SAN FELICE,NO2 (BIOSSIDO DI AZOTO),2019-01-11T08:01:00+01:00,2019-01-11T09:00:00+01:00,42.0,ug/m3
3,PORTA SAN FELICE,NO2 (BIOSSIDO DI AZOTO),2019-01-19T10:01:00+01:00,2019-01-19T11:00:00+01:00,42.0,ug/m3
4,PORTA SAN FELICE,NO2 (BIOSSIDO DI AZOTO),2019-01-27T06:01:00+01:00,2019-01-27T07:00:00+01:00,42.0,ug/m3
...,...,...,...,...,...,...
472614,PORTA SAN FELICE,NO (MONOSSIDO DI AZOTO),2022-01-07T22:01:00+01:00,2022-01-07T23:00:00+01:00,19.0,ug/m3
472615,GIARDINI MARGHERITA,PM10,2022-01-07T00:01:00+01:00,2022-01-08T00:00:00+01:00,21.0,ug/m3
472616,GIARDINI MARGHERITA,NO2 (BIOSSIDO DI AZOTO),2022-01-08T00:01:00+01:00,2022-01-08T01:00:00+01:00,21.0,ug/m3
472617,GIARDINI MARGHERITA,PM2.5,2022-01-07T00:01:00+01:00,2022-01-08T00:00:00+01:00,15.0,ug/m3


### Traffic Dataset

In [14]:
traffic_folder = os.path.join(datasets_folder,'traffic')

readings_df = merge_csv_to_dataframe(os.path.join(traffic_folder, 'readings'), sep=';')
accuracies_df = merge_csv_to_dataframe(os.path.join(traffic_folder, 'accuracies'), sep=';')

Merged 6 CSV files
Merged 6 CSV files


In [16]:
readings_df

,data,codice spira,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,...,stato,codimpsem,direzione,angolo,longitudine,latitudine,geopoint,ID_univoco_stazione_spira,Giorno della settimana,giorno settimana
0,2019-01-01,0.127 1.4 2 1,120,142,119,70,59,31,19,38,...,A,318.0,S,170.0,11.347424,44.500815,"44.5008152038968, 11.3474244254298",15.0,Martedì,NaN
1,2019-01-01,0.127 1.12 8 1,59,68,72,46,29,8,11,19,...,A,316.0,N,350.0,11.338970,44.495251,"44.4952505129043, 11.338970003537",2.0,Martedì,NaN
2,2019-01-01,0.127 1.15 6 1,37,31,33,27,13,8,15,16,...,A,324.0,S,168.0,11.343443,44.490027,"44.4900271601578, 11.3434427397635",7.0,Martedì,NaN
3,2019-01-01,0.127 1.18 1 1,22,23,20,16,14,22,34,35,...,A,327.0,S,163.0,11.338565,44.499223,"44.4992226472054, 11.338564528943",11.0,Martedì,NaN
4,2019-01-01,0.127 1.18 2 1,119,190,185,130,62,66,47,50,...,A,327.0,S,167.0,11.338515,44.499233,"44.4992331807033, 11.3385147346639",12.0,Martedì,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788560,2024-11-30,4.91 4.36 8 1,213,131,87,52,33,51,96,217,...,A,191.0,S,156.0,11.380487,44.478568,"44.4785683027768, 11.3804873332989",903.0,NaN,Sabato
1788561,2024-11-30,4.93 4.92 6 1,0,0,0,0,0,0,0,0,...,A,189.0,E,244.0,11.368549,44.484329,"44.4843292671686, 11.3685493809408",910.0,NaN,Sabato
1788562,2024-11-30,4.96 4.92 4 1,75,51,34,12,13,32,50,170,...,A,190.0,NE,341.0,11.371763,44.482901,"44.4829011144397, 11.371762568121",919.0,NaN,Sabato
1788563,2024-11-30,4.98 4.98 3 1,56,34,19,11,10,20,45,66,...,A,176.0,S,170.0,11.385260,44.468970,"44.4689700816957, 11.3852604232066",923.0,NaN,Sabato


In [17]:
accuracies_df

,data,codice spira,00:00-01:00,01:00-02:00,02:00-03:00,03:00-04:00,04:00-05:00,05:00-06:00,06:00-07:00,07:00-08:00,...,14:00-15:00,15:00-16:00,16:00-17:00,17:00-18:00,18:00-19:00,19:00-20:00,20:00-21:00,21:00-22:00,22:00-23:00,23:00-24:00
0,2019-01-14,4.30 4.96 4 1,100%,100%,100%,100%,100%,100%,100%,100%,...,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%
1,2019-01-14,4.31 4.29 6 1,100%,100%,100%,95%,58%,87%,100%,100%,...,83%,91%,100%,100%,91%,79%,95%,87%,95%,100%
2,2019-01-14,4.31 4.39 4 1,100%,100%,100%,100%,66%,83%,100%,100%,...,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%
3,2019-01-14,4.32 0.127 4 1,100%,100%,100%,100%,100%,100%,100%,100%,...,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%
4,2019-01-14,4.32 4.31 6 1,100%,100%,100%,100%,100%,100%,100%,100%,...,100%,100%,100%,100%,100%,100%,100%,100%,100%,100%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864591,2024-12-25,4.65 4.49 2 1,100%,100%,100%,100%,100%,83%,0%,50%,...,100%,100%,100%,91%,100%,100%,100%,100%,100%,100%
1864592,2024-12-25,4.65 4.65 4 1,100%,100%,100%,100%,100%,100%,100%,100%,...,100%,100%,100%,91%,100%,100%,100%,100%,100%,100%
1864593,2024-12-25,4.81 4.1 4 1,50%,50%,50%,50%,50%,50%,33%,33%,...,50%,50%,50%,50%,50%,50%,50%,50%,50%,50%
1864594,2024-12-25,4.81 4.1 4 2,100%,100%,100%,100%,100%,100%,100%,50%,...,100%,100%,100%,91%,100%,100%,100%,100%,100%,100%


### Weather Dataset

In [20]:
weather_folder = os.path.join(datasets_folder,'weather')

weather_df = merge_csv_to_dataframe(weather_folder)
weather_df

Merged 6 CSV files


,PragaTime,TAVG,PREC,RHAVG,RAD,W_SCAL_INT,W_VEC_DIR,W_VEC_INT,LEAFW,ET0
0,2019-01-01 00:00:00,1.0,0.0,92.3,0.0,0.5,208.7,0.3,0.0,0.0
1,2019-01-01 01:00:00,0.3,0.0,93.6,0.0,0.5,280.0,0.2,0.0,0.0
2,2019-01-01 02:00:00,0.7,0.0,91.7,0.0,1.1,158.1,1.0,0.0,0.0
3,2019-01-01 03:00:00,0.4,0.0,91.5,0.0,0.7,189.4,0.4,0.0,0.0
4,2019-01-01 04:00:00,1.0,0.0,86.3,0.0,0.7,172.9,0.6,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
52580,2024-12-30 20:00:00,6.2,0.0,72.1,0.0,2.0,268.9,1.6,0.0,0.0
52581,2024-12-30 21:00:00,5.6,0.0,76.1,0.0,2.5,258.0,2.4,0.0,0.0
52582,2024-12-30 22:00:00,5.7,0.0,75.3,0.0,2.8,254.1,2.7,0.0,0.0
52583,2024-12-30 23:00:00,5.1,0.0,76.1,0.0,2.8,256.7,2.7,0.0,0.0
